# group level for Social Prediction

*Yiyu Wang 2024 Feb*


In [1]:
import os

import glob
import nibabel as nib
import numpy as np
import pandas as pd
import copy

import nilearn
from nilearn.image import smooth_img, resample_to_img,new_img_like, math_img, concat_imgs, get_data
from nilearn import image
from nilearn import plotting
from nilearn.masking import apply_mask
from nilearn.input_data import NiftiMasker
from nilearn.glm.second_level import SecondLevelModel
from nilearn.reporting import get_clusters_table
from nilearn.glm import threshold_stats_img
from scipy.stats import norm

from nilearn.datasets import load_mni152_gm_mask,load_mni152_wm_mask,fetch_surf_fsaverage


import gzip
import math

import seaborn as sns
import matplotlib.pyplot as plt
from os.path import join


/Users/yiyuwang/anaconda3/envs/neuroimaging_env/lib/python3.7/site-packages/nilearn/__init__.py:67: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()
/Users/yiyuwang/anaconda3/envs/neuroimaging_env/lib/python3.7/site-packages/nilearn/input_data/__init__.py:23: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [2]:
# local directory set up
gm_mask_img = nib.load('masks/gm_mask_icbm152_brain.nii.gz')

subjects_list = pd.read_csv('Data/included_SocialPred_subjects.csv', header=None)
subjects_list = subjects_list[0].values.tolist()
sample_n = len(subjects_list)
print("subjects in this analysis:")
print(subjects_list)
print(f"**** n = {sample_n} *****" )


TR = .001
N_TR = 675
TR_Length = 0.8
TR_IN_MS = int(TR_Length/TR)

fwhm = 0


subjects in this analysis:
[152, 179, 154, 158, 173, 153, 159, 174, 162, 145, 143, 181, 144, 169, 146, 167, 161, 182, 147, 166, 160, 185, 170, 176, 151, 157, 171, 177, 150, 156]
**** n = 30 *****


In [3]:


beta_dir = 'fmri_results/1stLvl/'
second_level_res_dir = 'fmri_results/OneSampleT_3lvl/'

if not os.path.isdir(second_level_res_dir):
    os.mkdir(second_level_res_dir)


vmax = 5
cluster_thre = 30
p_val = 0.05
p_unc = norm.isf(p_val)

p001 = 0.001
p001_unc = norm.isf(p001)

alpha = 0.05

In [4]:
# create subject folders
for s in subjects_list:
    sub_output_dir = second_level_res_dir + f'/{s}/'
    if not os.path.isdir(sub_output_dir):
        os.makedirs(sub_output_dir)

In [5]:
# run obs social - obs pattern contrast:
for s in subjects_list:
    sub_output_dir = second_level_res_dir + f'/{s}/'
    # run contrast:
    print(f'running subject {s}')
    second_level_input = []
    obs_social_pattern_contrast = []

    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*z_score_*obs_Pattern*.nii.gz')
    second_level_input.append(file_name[0])
    obs_social_pattern_contrast.append(-1)


    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*z_score_*obs_Social*.nii.gz')
    second_level_input.append(file_name[0])
    obs_social_pattern_contrast.append(1)

    design_columns = ['intercept','obs_social_pattern']

    X = pd.concat([pd.Series(np.ones(len(second_level_input))), pd.Series(obs_social_pattern_contrast)], axis=1)
    X.columns= design_columns


    second_level_model = SecondLevelModel()
    second_level_model = second_level_model.fit(second_level_input,
                                                design_matrix=X)
    
    # get the beta:
    stats_name = 'effect_size'
    res = second_level_model.compute_contrast('obs_social_pattern',output_type=stats_name)
    nii_file_path = sub_output_dir + f'/obs_social_pattern_{stats_name}.nii.gz'
    nib.save(res, nii_file_path)


    plotting.plot_design_matrix(X, output_file=join(second_level_res_dir, f'design_matrix.png'))



running subject 152


/Users/yiyuwang/anaconda3/envs/neuroimaging_env/lib/python3.7/site-packages/nilearn/glm/regression.py:207: RuntimeWarning: divide by zero encountered in true_divide
  self.whitened_design.shape[0] - self.whitened_design.shape[1]
/Users/yiyuwang/anaconda3/envs/neuroimaging_env/lib/python3.7/site-packages/nilearn/glm/regression.py:207: RuntimeWarning: invalid value encountered in true_divide
  self.whitened_design.shape[0] - self.whitened_design.shape[1]


running subject 179
running subject 154
running subject 158
running subject 173
running subject 153
running subject 159
running subject 174
running subject 162
running subject 145
running subject 143
running subject 181
running subject 144
running subject 169
running subject 146
running subject 167
running subject 161
running subject 182
running subject 147
running subject 166
running subject 160
running subject 185
running subject 170
running subject 176
running subject 151
running subject 157
running subject 171
running subject 177
running subject 150
running subject 156


In [6]:
# run anova:

for s in subjects_list:
    sub_output_dir = second_level_res_dir + f'/{s}/'
    # run contrast:
    print(f'running subject {s}')
    
    file_list = []
    main_condition_contrast, main_PE_contrast, interaction_contrast = [], [], []


    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*z_score_*fb_Social_PE*.nii.gz')
    file_list.append(file_name[0])
    main_condition_contrast.append(1)
    main_PE_contrast.append(1) 
    interaction_contrast.append(1)

    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*z_score_*fb_Social_Congruent*.nii.gz')
    file_list.append(file_name[0])
    main_condition_contrast.append(1)
    main_PE_contrast.append(-1) 
    interaction_contrast.append(-1)

    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*z_score_*fb_Pattern_PE*.nii.gz')
    file_list.append(file_name[0])
    main_condition_contrast.append(-1)
    main_PE_contrast.append(1) 
    interaction_contrast.append(-1)

    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*z_score_*fb_Pattern_Congruent*.nii.gz')
    file_list.append(file_name[0])
    main_condition_contrast.append(-1)
    main_PE_contrast.append(-1) 
    interaction_contrast.append(1)


    second_level_input = file_list

    design_columns = ['intercept','main_condition','main_PE','interaction']

    X = pd.concat([pd.Series(np.ones(len(file_list))), 
                pd.Series(main_condition_contrast), 
                pd.Series(main_PE_contrast), 
                pd.Series(interaction_contrast)], axis=1)
    X.columns = design_columns

    second_level_input = file_list
    second_level_model = SecondLevelModel(mask_img=gm_mask_img)
    second_level_model = second_level_model.fit(second_level_input, design_matrix=X)
    stats_name = 'effect_size'
    
    res = second_level_model.compute_contrast('interaction',output_type=stats_name)
    nii_file_path = sub_output_dir + f'/interaction_{stats_name}.nii.gz'
    nib.save(res, nii_file_path)

    res = second_level_model.compute_contrast('main_PE',output_type=stats_name)
    nii_file_path = sub_output_dir + f'/main_PE_{stats_name}.nii.gz'
    nib.save(res, nii_file_path)

    res = second_level_model.compute_contrast('main_condition',output_type=stats_name)
    nii_file_path = sub_output_dir + f'/main_condition_{stats_name}.nii.gz'
    nib.save(res, nii_file_path)



running subject 152
running subject 179
running subject 154
running subject 158
running subject 173
running subject 153
running subject 159
running subject 174
running subject 162
running subject 145
running subject 143
running subject 181
running subject 144
running subject 169
running subject 146
running subject 167
running subject 161
running subject 182
running subject 147
running subject 166
running subject 160
running subject 185
running subject 170
running subject 176
running subject 151
running subject 157
running subject 171
running subject 177
running subject 150
running subject 156


In [8]:
for s in subjects_list:
    sub_output_dir = second_level_res_dir + f'/{s}/'
    # run contrast:
    print(f'running subject {s}')   
    social_PE_congruent_contrast, pattern_PE_congruent_contrast = [], []
    social_PE_contruent_input, pattern_PE_contruent_input = [], []   
    
    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*beta*fb_Social_PE*.nii.gz')
    social_PE_contruent_input.append(file_name[0])
    social_PE_congruent_contrast.append(1)

    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*beta*fb_Social_Congruent*.nii.gz')
    social_PE_contruent_input.append(file_name[0])
    social_PE_congruent_contrast.append(-1)

    design_columns = ['intercept','fb_social_PE_congruent']

    X = pd.concat([pd.Series(np.ones(len(social_PE_contruent_input))), 
                pd.Series(social_PE_congruent_contrast)], axis=1)
    X.columns = design_columns

    second_level_input = social_PE_contruent_input
    second_level_model = SecondLevelModel(mask_img=gm_mask_img)
    second_level_model = second_level_model.fit(second_level_input, design_matrix=X)

    stats_name = 'effect_size'
    res = second_level_model.compute_contrast('fb_social_PE_congruent',output_type=stats_name)

    nii_file_path = sub_output_dir + f'/fb_social_PE_congruent_{stats_name}.nii.gz'
    nib.save(res, nii_file_path)


running subject 152
running subject 179
running subject 154
running subject 158
running subject 173
running subject 153
running subject 159
running subject 174
running subject 162
running subject 145
running subject 143
running subject 181
running subject 144
running subject 169
running subject 146
running subject 167
running subject 161
running subject 182
running subject 147
running subject 166
running subject 160
running subject 185
running subject 170
running subject 176
running subject 151
running subject 157
running subject 171
running subject 177
running subject 150
running subject 156


In [9]:
for s in subjects_list:
    sub_output_dir = second_level_res_dir + f'/{s}/'
    # run contrast:
    print(f'running subject {s}')     
    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*beta*fb_Pattern_PE*.nii.gz')
    pattern_PE_contruent_input.append(file_name[0])
    pattern_PE_congruent_contrast.append(1)

    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*beta*fb_Pattern_Congruent*.nii.gz')
    pattern_PE_contruent_input.append(file_name[0])
    pattern_PE_congruent_contrast.append(-1)

    design_columns = ['intercept','fb_pattern_PE_congruent']

    X = pd.concat([pd.Series(np.ones(len(pattern_PE_contruent_input))), 
                pd.Series(pattern_PE_congruent_contrast)], axis=1)
    X.columns = design_columns

    second_level_input = pattern_PE_contruent_input
    second_level_model = SecondLevelModel()
    second_level_model = second_level_model.fit(second_level_input, design_matrix=X)

    stats_name = 'effect_size'
    res = second_level_model.compute_contrast('fb_pattern_PE_congruent',output_type=stats_name)
    nii_file_path = sub_output_dir + f'/fb_pattern_PE_congruent_{stats_name}.nii.gz'
    nib.save(res, nii_file_path)


running subject 152
running subject 179
running subject 154
running subject 158
running subject 173
running subject 153
running subject 159
running subject 174
running subject 162
running subject 145
running subject 143
running subject 181
running subject 144
running subject 169
running subject 146
running subject 167
running subject 161
running subject 182
running subject 147
running subject 166
running subject 160
running subject 185
running subject 170
running subject 176
running subject 151
running subject 157
running subject 171
running subject 177
running subject 150
running subject 156


In [10]:
# fb_PE_congruent


for s in subjects_list:
    sub_output_dir = second_level_res_dir + f'/{s}/'
    # run contrast:
    print(f'running subject {s}')  

    PE_congruent_input, PE_congruent_contrast = [], []
       
    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*beta_video-fb_Pattern_PE*.nii.gz')
    PE_congruent_input.append(file_name[0])
    PE_congruent_contrast.append(1)

    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*beta_video-fb_Social_PE*.nii.gz')
    PE_congruent_input.append(file_name[0])
    PE_congruent_contrast.append(1)

    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*beta_video-fb_Pattern_Congruent*.nii.gz')
    PE_congruent_input.append(file_name[0])
    PE_congruent_contrast.append(-1)

    file_name = glob.glob(beta_dir + f'{s}/sub-{s}*beta_video-fb_Social_Congruent*.nii.gz')
    PE_congruent_input.append(file_name[0])
    PE_congruent_contrast.append(-1)

    design_columns = ['intercept','fb_PE_congruent']

    X = pd.concat([pd.Series(np.ones(len(PE_congruent_input))), 
                pd.Series(PE_congruent_contrast)], axis=1)
    X.columns = design_columns

    second_level_input = PE_congruent_input
    second_level_model = SecondLevelModel()
    second_level_model = second_level_model.fit(second_level_input, design_matrix=X)

    stats_name = 'effect_size'
    res = second_level_model.compute_contrast('fb_PE_congruent',output_type=stats_name)
    nii_file_path = sub_output_dir + f'/fb_PE_congruent_{stats_name}.nii.gz'
    nib.save(res, nii_file_path)


running subject 152
running subject 179
running subject 154
running subject 158
running subject 173
running subject 153
running subject 159
running subject 174
running subject 162
running subject 145
running subject 143
running subject 181
running subject 144
running subject 169
running subject 146
running subject 167
running subject 161
running subject 182
running subject 147
running subject 166
running subject 160
running subject 185
running subject 170
running subject 176
running subject 151
running subject 157
running subject 171
running subject 177
running subject 150
running subject 156


In [11]:
# group level:

contrast_list = ['obs_social_pattern', 'interaction', 'main_PE', 'main_condition', 'fb_social_PE_congruent', 'fb_pattern_PE_congruent', 'fb_PE_congruent']
for contrast in contrast_list:
    print(f'running group level contrast: {contrast}')
    group_level_input = glob.glob(second_level_res_dir + f'/*/{contrast}_effect_size.nii.gz')

    design_columns = ['intercept']
    X = pd.DataFrame(np.ones(len(group_level_input)), columns = design_columns)

    group_level_model = SecondLevelModel()
    group_level_model = group_level_model.fit(group_level_input, design_matrix=X)
    group_res = group_level_model.compute_contrast('intercept',output_type='all')
    for stats_name in group_res.keys():
        res = group_res[stats_name]
        nii_file_path = second_level_res_dir + f'/{contrast}_{stats_name}.nii.gz'
        nib.save(res, nii_file_path)
    

running group level contrast: obs_social_pattern
running group level contrast: interaction
running group level contrast: main_PE
running group level contrast: main_condition
running group level contrast: fb_social_PE_congruent
running group level contrast: fb_pattern_PE_congruent
running group level contrast: fb_PE_congruent


In [12]:
for contrast in contrast_list:
    print(f'contrast: {contrast}')
    z_map = nib.load(second_level_res_dir + f'/{contrast}_z_score.nii.gz')
    fwe_corrected_img, fwe = threshold_stats_img(z_map, mask_img=gm_mask_img, alpha=0.05, cluster_threshold = 50, height_control='bonferroni')
    print(f'FWE corrected p value is {fwe}')
    print(f'max value is {np.max(z_map.get_fdata())}')
    print(f'min value is {np.min(z_map.get_fdata())}')


contrast: obs_social_pattern


/Users/yiyuwang/anaconda3/envs/neuroimaging_env/lib/python3.7/site-packages/nilearn/_utils/param_validation.py:73: UserWarning: The given float value must not exceed 4.730902651176177. But, you have given threshold=5.149072733780298.
  f"The given float value must not exceed {value_check}. "


FWE corrected p value is 5.149072733780298
max value is 4.730902651176177
min value is -3.419279368772976
contrast: interaction
FWE corrected p value is 5.149072733780298
max value is 6.092518757011305
min value is -5.9509785491284575
contrast: main_PE
FWE corrected p value is 5.149072733780298
max value is 4.941261744419585
min value is -7.093916147487632
contrast: main_condition


/Users/yiyuwang/anaconda3/envs/neuroimaging_env/lib/python3.7/site-packages/nilearn/_utils/param_validation.py:73: UserWarning: The given float value must not exceed 5.051749669722499. But, you have given threshold=5.149072733780298.
  f"The given float value must not exceed {value_check}. "


FWE corrected p value is 5.149072733780298
max value is 5.051749669722499
min value is -3.5689180512265213
contrast: fb_social_PE_congruent


/Users/yiyuwang/anaconda3/envs/neuroimaging_env/lib/python3.7/site-packages/nilearn/_utils/param_validation.py:73: UserWarning: The given float value must not exceed 5.076379033339101. But, you have given threshold=5.149072733780298.
  f"The given float value must not exceed {value_check}. "


FWE corrected p value is 5.149072733780298
max value is 4.683319935802479
min value is -5.076379033339101
contrast: fb_pattern_PE_congruent
FWE corrected p value is 5.149072733780298
max value is 11.908241687996963
min value is -10.958846449543787
contrast: fb_PE_congruent
FWE corrected p value is 5.149072733780298
max value is 3.517386066663037
min value is -3.7917926876932437


/Users/yiyuwang/anaconda3/envs/neuroimaging_env/lib/python3.7/site-packages/nilearn/_utils/param_validation.py:73: UserWarning: The given float value must not exceed 3.7917926876932437. But, you have given threshold=5.149072733780298.
  f"The given float value must not exceed {value_check}. "
